In [1]:
import pandas as pd
import random
from rdflib.namespace import RDF, RDFS, FOAF, XSD, URIRef
from rdflib import Graph
import pandas as pd
from rdflib import Namespace
from rdflib import Literal

In [2]:
authorsDF = pd.read_csv('data/authors.csv')
authorsPapersDF = pd.read_csv('data/authorsPapers.csv')
chairsDF = pd.read_csv('data/chairs.csv')
conferencesDF = pd.read_csv('data/conferences.csv')
conferenceProceedingsDF = pd.read_csv('data/conferenceProceedings.csv')
domainDF = pd.read_csv('data/domain.csv')
editorsDF = pd.read_csv('data/editors.csv')
journalsDF = pd.read_csv('data/journals.csv')
journalVolumesDF = pd.read_csv('data/journalVolumes.csv')
papersDF = pd.read_csv('data/papers.csv')
proceedingsDF = pd.read_csv('data/proceedings.csv')
reviewersDF = pd.read_csv('data/reviewers.csv')
reviewsDF = pd.read_csv('data/reviews.csv')
subjectsProceedingsVolumesDF = pd.read_csv('data/subjectsProceedingsVolumes.csv')
subjectsPapersDF = pd.read_csv('data/subjectsPapers.csv')
volumesDF = pd.read_csv('data/volumes.csv')

In [3]:
graph = Graph()
lab = Namespace("http://sdmlab.org/")

In [4]:
graph.bind('lab',lab)

### Paper -- [paperTitle] --> String 

In [5]:
papersDF.head(1)

,paperId,paperUrl,conferenceJournalId,conferenceJournalTitle,paperTitle,paperAbstract,paperType,proceedingsVolumeIds
0,p0,133bcd7488a3c07cb0f493a87564c30e5433768c,j0,Nature Biotechnology,"Reproducible, interactive, scalable and extens...",Abstract,fullPaper,jv0


In [6]:
graph.add((lab.Paper, lab.paperTitle, XSD.string))
for k in range(len(papersDF['paperId'])):
    graph.add((URIRef(lab+papersDF['paperId'][k]), lab.paperTitle, Literal(papersDF['paperTitle'][k])))

### Paper -- [paperAbstract] --> String

In [7]:
graph.add((lab.Paper, lab.paperAbstract, XSD.string))
for k in range(len(papersDF['paperId'])):
    graph.add((URIRef(lab+papersDF['paperId'][k]), lab.paperAbstract, Literal(papersDF['paperAbstract'][k])))

### Paper -- [isSubmittedTo] --> String

In [8]:
graph.add((lab.Paper, lab.isSubmittedTo, lab.Venue))
for k in range(len(papersDF['paperId'])):
    graph.add((URIRef(lab+papersDF['paperId'][k]), lab.isSubmittedTo, URIRef(lab+papersDF['conferenceJournalId'][k])))

### Paper -- [isPublishedInProc] --> Proceedings & Paper -- [isPublishedInVol] --> Volume

In [9]:
graph.add((lab.Paper, lab.isPublishedInProc, lab.Proceeding))
graph.add((lab.FinalPaper, lab.isPublishedInVol, lab.Volume))
for k in range(len(papersDF['paperId'])):
    if papersDF['conferenceJournalId'][k][0] == 'c':
        graph.add((URIRef(lab+papersDF['paperId'][k]), lab.isPublishedInProc, URIRef(lab+papersDF['proceedingsVolumeIds'][k])))
    else:
        graph.add((URIRef(lab+papersDF['paperId'][k]), lab.isPublishedInVol, URIRef(lab+papersDF['proceedingsVolumeIds'][k])))

### Author -- [authorName] --> String 

In [10]:
authorsDF.head(1)

,authorId,authorUrl,authorName
0,a0,6391996,Evan Bolyen


In [11]:
graph.add((lab.Author, lab.authorName, XSD.string))
for k in range(len(authorsDF['authorId'])):
    graph.add((URIRef(lab+authorsDF['authorId'][k]), lab.authorName, Literal(authorsDF['authorName'][k])))

### Author -- [writes] --> Paper

In [12]:
authorsPapersDF.head(1)

,authorUrl,paperId,authorId,authorName,paperUrl,conferenceJournalId,conferenceJournalTitle,paperTitle,paperAbstract,paperType,proceedingsVolumeIds
0,6391996,p0,a0,Evan Bolyen,133bcd7488a3c07cb0f493a87564c30e5433768c,j0,Nature Biotechnology,"Reproducible, interactive, scalable and extens...",Abstract,fullPaper,jv0


In [13]:
graph.add((lab.Author, lab.writes, lab.Paper))   
for k in range(len(authorsPapersDF['authorId'])):
    graph.add((URIRef(lab+authorsPapersDF['authorId'][k]), lab.writes, URIRef(lab+authorsPapersDF['paperId'][k])))
    graph.add((URIRef(lab+authorsPapersDF['authorId'][k]), RDF.type, lab.Author))
    ptype = authorsPapersDF['paperType'][k]
    if ptype == "fullPaper":
        graph.add((URIRef(lab+authorsPapersDF['paperId'][k]), RDF.type, lab.FullPaper))
    elif ptype == "poster":
        graph.add((URIRef(lab+authorsPapersDF['paperId'][k]), RDF.type, lab.Poster))

### Paper -- [hasPaperDomain] --> Domain

In [14]:
subjectsPapersDF.head(1)

,domainName,paperId,domainId,paperUrl,conferenceJournalId,conferenceJournalTitle,paperTitle,paperAbstract,paperType,proceedingsVolumeIds
0,Engineering,p0,d0,133bcd7488a3c07cb0f493a87564c30e5433768c,j0,Nature Biotechnology,"Reproducible, interactive, scalable and extens...",Abstract,fullPaper,jv0


In [15]:
graph.add((lab.Paper, lab.hasPaperDomain, lab.Domain))
for k in range(len(subjectsPapersDF['paperId'])):
    graph.add((URIRef(subjectsPapersDF['paperId'][k]), lab.hasPaperDomain, URIRef(subjectsPapersDF['domainId'][k])))

### Chair -- [manages] --> Conference

In [16]:
chairsDF.head(1)

,conferenceId,chairId,authorId
0,c79,chair0,a2516


In [17]:
graph.add((lab.Chair, lab.manages, lab.Conference))
for k in range(len(chairsDF['conferenceId'])):
    graph.add((URIRef(lab+chairsDF['authorId'][k]), lab.manages, URIRef(lab+chairsDF['conferenceId'][k])))

### Editor -- [handles] --> [Journal]

In [18]:
editorsDF.head(1)

,journalId,editorId,authorId
0,j0,editor0,a116


In [19]:
graph.add((lab.Editor, lab.handles, lab.Journal))
for k in range(len(editorsDF['journalId'])):
    graph.add((URIRef(lab+editorsDF['authorId'][k]), lab.handles, URIRef(lab+editorsDF['journalId'][k])))

### Conference -- [conferenceTitle] --> String 

In [20]:
conferencesDF.head(1)

,conferenceId,conferenceUrl,conferenceTitle,conferenceType,conferenceCity,conferenceYear,conferenceProceedingIds
0,c0,fc0a208c-acb7-47dc-a0d4-af8190e21d29,International Conference on Machine Learning,mainConference,New York,2023,cp0


In [21]:
graph.add((lab.Conference, lab.conferenceTitle, XSD.string))
for k in range(len(conferencesDF['conferenceId'])):
    graph.add((URIRef(lab+conferencesDF['conferenceId'][k]), lab.conferenceTitle, Literal(conferencesDF['conferenceTitle'][k])))

### Conference -- [conferenceCity] --> String 

In [22]:
graph.add((lab.Conference, lab.conferenceCity, XSD.string))
for k in range(len(conferencesDF['conferenceId'])):
    graph.add((URIRef(lab+conferencesDF['conferenceId'][k]), lab.conferenceCity, Literal(conferencesDF['conferenceCity'][k])))

### Conference -- [conferenceYear] --> int 

In [23]:
graph.add((lab.Conference, lab.conferenceYear, XSD.int))
for k in range(len(conferencesDF['conferenceId'])):
    graph.add((URIRef(lab+conferencesDF['conferenceId'][k]), lab.conferenceYear, Literal(conferencesDF['conferenceYear'][k])))

### Conference -- [inProc] --> Proceedings

In [24]:
conferenceProceedingsDF.head(1)

,conferenceId,conferenceUrl,conferenceTitle,conferenceType,conferenceCity,conferenceYear,conferenceProceedingIds,proceedingName
0,c0,fc0a208c-acb7-47dc-a0d4-af8190e21d29,International Conference on Machine Learning,mainConference,New York,2023,cp0,proceeding0


In [25]:
graph.add((lab.Conference, lab.inProc, lab.Proceedings))
for k in range(len(conferenceProceedingsDF['conferenceId'])):
    graph.add((URIRef(lab+conferenceProceedingsDF['conferenceId'][k]), lab.inProc, URIRef(lab+conferenceProceedingsDF['conferenceProceedingIds'][k])))

### Journal -- [journalTitle] --> String

In [26]:
journalsDF.head(1)

,journalId,journalUrl,journalTitle,journalVolumeIds
0,j0,458166b3-de17-4bf3-bbbb-e53782de2f0f,Nature Biotechnology,jv0


In [27]:
graph.add((lab.Journal, lab.journalTitle, XSD.string))
for k in range(len(journalsDF['journalId'])):
    graph.add((URIRef(lab+journalsDF['journalId'][k]), lab.journalTitle, Literal(journalsDF['journalTitle'][k])))

### Journal -- [inVol] --> Volume

In [28]:
journalVolumesDF.head(1)

,journalId,journalUrl,journalTitle,journalVolumeIds,volumeName,volumeYear
0,j0,458166b3-de17-4bf3-bbbb-e53782de2f0f,Nature Biotechnology,jv0,journal0,2023


In [29]:
graph.add((lab.Journal, lab.inVol, lab.Volume))
for k in range(len(journalVolumesDF['journalId'])):
    graph.add((URIRef(lab+journalVolumesDF['journalId'][k]), lab.inVol, URIRef(lab+journalVolumesDF['journalVolumeIds'][k])))

### Proceedings -- [proceedingName] --> String 

In [30]:
proceedingsDF.head(1)

,proceedingId,proceedingName
0,cp0,proceeding0


In [31]:
graph.add((lab.Proceedings, lab.procName, XSD.string))
for k in range(len(proceedingsDF['proceedingId'])):
    graph.add((URIRef(lab+proceedingsDF['proceedingId'][k]), lab.procName, Literal(proceedingsDF['proceedingName'][k])))

### Volume -- [volName] --> String 

In [32]:
volumesDF.head(1)

,volumeId,volumeName,volumeYear
0,jv0,journal0,2023


In [33]:
graph.add((lab.Volume, lab.volName, XSD.string))
for k in range(len(volumesDF['volumeId'])):
    graph.add((URIRef(lab+volumesDF['volumeId'][k]), lab.volName, Literal(volumesDF['volumeName'][k])))

### Volume -- [volYear] --> int

In [34]:
graph.add((lab.Volume, lab.volYear, XSD.int))
for k in range(len(volumesDF['volumeId'])):
    graph.add((URIRef(lab+volumesDF['volumeId'][k]), lab.volYear, Literal(volumesDF['volumeYear'][k])))

### Proceeding -- [hasProcDomain] --> Domain & Volume -- [hasVolDomain] --> Domain

In [35]:
subjectsProceedingsVolumesDF.head(2)

,conferenceJournalId,proceedingsVolumeIds,domainId
0,j0,jv0,d0
1,j0,jv0,d1


In [36]:
graph.add((lab.Proceeding, lab.hasProcDomain, lab.Domain))
graph.add((lab.Volume, lab.hasVolDomain, lab.Domain))
for k in range(len(subjectsProceedingsVolumesDF['proceedingsVolumeIds'])):
    curr_confJour_keyword = subjectsProceedingsVolumesDF['domainId'][k]
    if subjectsProceedingsVolumesDF['proceedingsVolumeIds'][k][0] == 'c':
        graph.add((URIRef(lab+subjectsProceedingsVolumesDF['proceedingsVolumeIds'][k]), lab.hasProcDomain, URIRef(lab+subjectsProceedingsVolumesDF['domainId'][k])))
    else:
        graph.add((URIRef(lab+subjectsProceedingsVolumesDF['proceedingsVolumeIds'][k]), lab.hasVolDomain, URIRef(lab+subjectsProceedingsVolumesDF['domainId'][k])))

### Domain -- [keywords] --> String

In [37]:
domainDF.head(1)

,domainId,domainName
0,d0,Engineering


In [38]:
graph.add((lab.Domain, lab.keywords, XSD.string))
for k in range(len(domainDF['domainId'])):
    graph.add((URIRef(lab+domainDF['domainId'][k]), lab.keywords, Literal(domainDF['domainName'][k])))

### Review-- [reviewText] --> String

In [39]:
reviewsDF.head(1)

,reviewId,paperId,reviewText,reviewDecision,paperTitle
0,r0,p0,Review Comment,1,"Reproducible, interactive, scalable and extens..."


In [40]:
graph.add((lab.Review, lab.reviewText, XSD.string))
for k in range(len(reviewsDF['reviewId'])):
    graph.add((URIRef(lab+reviewsDF['reviewId'][k]), lab.reviewText, Literal(reviewsDF['reviewText'][k])))

### ReviewText -- [decision] --> boolean 

In [41]:
graph.add((lab.Review, lab.decision, XSD.boolean))
for k in range(len(reviewsDF['reviewId'])):
    graph.add((URIRef(lab+reviewsDF['reviewId'][k]), lab.decision, Literal(reviewsDF['reviewDecision'][k])))

### Reviewer -- [gives] --> Review

In [42]:
reviewersDF.head(1)

,rId,paperId,authorId,authorUrl,authorName,paperUrl,conferenceJournalId,conferenceJournalTitle,paperTitle,paperAbstract,paperType,proceedingsVolumeIds
0,r0,p0,a6763,2149265778,Jing Wu,133bcd7488a3c07cb0f493a87564c30e5433768c,j0,Nature Biotechnology,"Reproducible, interactive, scalable and extens...",Abstract,fullPaper,jv0


In [43]:
graph.add((lab.Reviewer, lab.gives, lab.Review))
for k in range(len(reviewersDF['rId'])):
    graph.add((URIRef(lab+reviewersDF['authorId'][k]), lab.gives, URIRef(lab+reviewersDF['conferenceJournalId'][k])))

### Review -- [isGivenTo] --> Paper

In [45]:
reviewsDF.head(1)

,reviewId,paperId,reviewText,reviewDecision,paperTitle
0,r0,p0,Review Comment,1,"Reproducible, interactive, scalable and extens..."


In [46]:
graph.add((lab.Review, lab.isGivenTo, lab.Paper))
for k in range(len(reviewsDF['reviewId'])):
    graph.add((URIRef(lab+reviewsDF['reviewId'][k]), lab.isGivenTo, URIRef(lab+reviewsDF['paperId'][k])))

### TTL File

In [47]:
graph.serialize(destination='data/abox.ttl', format="ttl")

<Graph identifier=N4fc62f513c6245f3aa8b64ba0ada673a (<class 'rdflib.graph.Graph'>)>